In [ ]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

# Import

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')

import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation

# Load the required Sionna components
from sionna.nr.my_abc import *

# Load model weight

In [3]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,48,14,18])
_model(inputs)
_model.summary()

#load_weights(_model, '/content/drive/MyDrive/Pusch_data/Model_weights/model_weight_FULL_RB_epoch_40.pkl')
# load_weights(_model, '../model_weight_FULL_RB_epoch_40.pkl')
load_weights(_model, '../weight_4RB_UMI_dynamic_config.pkl')

Model: "custom_neural_receiver"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  20864     
                                                                 
 residual_block (ResidualBl  multiple                  639232    
 ock)                                                            
                                                                 
 residual_block_1 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_2 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_3 (Residual  multiple                  639232    
 Block)                                     

# Set up config

In [ ]:
"""test setup"""
batch_size = 8

"""nrb config setup"""
RB_start = 0
NRB = 162
PCI = 443
RNTI = 40035
MCS = 8

"""channel setup"""
no = 2.
CDL_model = 'A'
delay_spread = 50
speed = 1

## create channel

In [7]:
channel_model = CDL(model = CDL_model,
                            delay_spread = delay_spread*1e-9,
                            carrier_frequency = CARRIER_FREQUENCY,
                            ut_array = Ue_Antenna,
                            bs_array = Gnb_AntennaArray,
                            direction = 'uplink',
                            min_speed = speed,
                            max_speed = speed)

## create 4RB samples

In [9]:
"""default config is 4RB"""
sysCfg = SystemConfig()
ueCfg = UeConfig()
myCfg = MyConfig(sysCfg, [ueCfg])
puschCfg = MyPUSCHConfig(myCfg)
# puschCfg.show() # uncomment for detail

In [10]:
simulator = MySimulator(puschCfg)
channel = OFDMChannel(channel_model=channel_model, resource_grid=simulator.resource_grid,
                                    add_awgn=False, normalize_channel=True, return_channel=True)

In [11]:
b, c, y, x ,h = simulator.sim(batch_size, channel, no, return_tx_iq=True, return_channel=True)
r = simulator.ref(batch_size)

## create N-RB samples

In [12]:
sysCfg = SystemConfig(
                    NCellId = PCI,
                    FrequencyRange = 1,
                    BandWidth = 60,
                    Numerology = 1,
                    CpType = 0,
                    NTxAnt = 1,
                    NRxAnt = 8,
                    BwpNRb = 162,
                    BwpRbOffset = 0,
                    harqProcFlag = 0,
                    nHarqProc = 1,
                    rvSeq = 0
                )
ueCfg = UeConfig(
                TransformPrecoding = 0,
                Rnti = RNTI,
                nId = PCI,
                CodeBookBased = 0,
                DmrsPortSetIdx = [0],
                NLayers = 1,
                NumDmrsCdmGroupsWithoutData = 2,
                Tpmi = 0,
                FirstSymb = 0,
                NPuschSymbAll = 14,
                RaType = 1,
                FirstPrb = RB_start,
                NPrb = NRB,
                FrequencyHoppingMode = 0,
                McsTable = 0,
                Mcs = MCS,
                ILbrm = 0,
                nScId = 0,
                NnScIdId = PCI,
                DmrsConfigurationType = 0,
                DmrsDuration = 1,
                DmrsAdditionalPosition = 1,
                PuschMappingType = 0,
                DmrsTypeAPosition = 3,
                HoppingMode = 0,
                NRsId = 0,
                Ptrs = 0,
                ScalingFactor = 0,
                OAck = 0,
                IHarqAckOffset = 11,
                OCsi1 = 0,
                ICsi1Offset = 7,
                OCsi2 = 0,
                ICsi2Offset = 0,
                NPrbOh = 0,
                nCw = 1,
                TpPi2Bpsk = 0
            )
myCfg = MyConfig(sysCfg, [ueCfg])
puschCfg = MyPUSCHConfig(myCfg)
# puschCfg.show() # uncomment for detail

In [13]:
simulator = MySimulator(puschCfg)
channel = OFDMChannel(channel_model=channel_model, resource_grid=simulator.resource_grid,
                                    add_awgn=False, normalize_channel=True, return_channel=True)

In [14]:
b, c, y, x ,h = simulator.sim(batch_size, channel, no, return_tx_iq=True, return_channel=True)
r = simulator.ref(batch_size)

# Evaluate

In [15]:
"""Evaluate"""
preds = predict(_model, y, r)
c_pred = tf.reshape(preds, [preds.shape[0], 1, c.shape[-1]])
b_hat, crc = simulator.TB_Decode(c_pred)
# loss_cal(c_pred, c)
compute_ber(b, b_hat), crc

(<tf.Tensor: shape=(), dtype=float64, numpy=0.003543534856812265>,
 <tf.Tensor: shape=(8, 1), dtype=bool, numpy=
 array([[ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True]])>)

In [16]:
h_est, x_hat, llr_det, b_hat, crc = simulator.rec(y)
compute_ber(b, b_hat), crc

(<tf.Tensor: shape=(), dtype=float64, numpy=0.13842385015909747>,
 <tf.Tensor: shape=(8, 1), dtype=bool, numpy=
 array([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False]])>)